Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Training a GNN to do Fraud Detection on Graphcore IPUs using your own dataset with PyTorch Geometric

TODO: Everything in this section

TODO: Update links:

[![Run on Gradient](../../gradient-badge.svg)](https://console.paperspace.com/github/<runtime-repo>?machine=Free-IPU-POD4&container=<dockerhub-image>&file=<path-to-file-in-repo>)  [![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

>
> We aim to have our notebook app demos to be focused on what the user is trying to
> do. To help you do this correctly please read [our user personnas](https://graphcore.atlassian.net/wiki/spaces/PM/pages/3157131517/Notebook+personas#Ellie%3A-The-Data-Scientist%2C-Business-Analysis%2C-Consultant),
and when in doubt ask yourself "does that person care about this?".
> To support that the first paragraph will contain all the key information, to
> help users rapidly identify if this is the right notebook for them to go
> though, based on:
>
> - The task/business problem they are trying to solve,
> - The features that are used (Focus on big picture Deep learning features - e.g.
>  Distributed training, not I/O overlap).
>
> To achieve this, each notebook should start with the following 3 paragraphs
> (detailed in the next three comments):
>
> - a table highlighting what we are going to do
> - a very short intro (3-5 sentences)
> - clear "steps to resolution" (bullet points stating what the user will have
>    to do to tackle their problem on the IPU - these need to reflect the notebook,
>    and be as simple as possible)
> - links to additional related resources.
>

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
|   GNNs   |  Fraud detection  | ? | ? | Training, evaluation | recommended: 16XX (min: 4X) | 20Xmn (X1h20mn)   |

>
>
> Start with a short introduction to the notebook. [suggested 3-5 sentences]
>
> This intro should focus on the problem you are fixing, and not on any IPU specific
> or framework specific features. The mindset is that anything that is non-standard
> is a barrier to entry, and will risk the user giving up.
>
> This short introduction should be followed by a clear bullet point summary of
> the steps of the demo. Each outcome should be of the form:
> - what the user will do (active verb) [and (optionally) how they do
>   it]. Jargon, if any, goes to the end of the bullet point.

In this demo, you will learn how to:

- Turn tabular transaction data into a PyTorch Geometric dataset
- Select a model suitable for the task of predicting fraudulent transactions
- Train the model on Graphcore IPUs
- Run validation on the trained model

This notebook assumes some familiarity with PopTorch as well as PyTorch Geometric (PyG). For additional resources please consult:
* [PopTorch Documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/index.html)
* [PopTorch Examples and Tutorials](https://docs.graphcore.ai/en/latest/examples.html#pytorch)
* [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
* [PopTorch Geometric Documentation](https://docs.graphcore.ai/projects/poptorch-geometric-user-guide/en/latest/index.html)

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

In [3]:
# Make imported python modules automatically reload when the files are changed
# needs to be before the first import.
%load_ext autoreload
%autoreload 2
# TODO: remove at the end of notebook development

## Environment setup

[![Run on Gradient](../../gradient-badge.svg)](TODO)

The best way to try this demo is on Paperspace Gradient's cloud IPUs. To use on other hardware
make sure that you have the Poplar SDK enabled with the latest PopTorch Geometric installed.

In [4]:
%pip install -q -r requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


To improve your experience we read some configuration related to the environment you are running the notebook in.

In [5]:
import os

number_of_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 16))
pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod16")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/")

# TODO Remove default
dataset_directory = os.getenv("DATASETS_DIR", "~")
checkpoint_directory = os.getenv("CHECKPOINT_DIR")

## Loading tabular data into PyTorch Geometric

In [6]:
import torch

### Getting the dataset

TODO: Using https://www.kaggle.com/c/ieee-fraud-detection/data

TODO: Run a script to download and tidy data?

In [202]:
import os.path as osp
import pandas as pd

raw_dataset_path = osp.join(dataset_directory, "ieee-fraud-detection", "raw")

train_transaction_path = osp.join(raw_dataset_path, "train_transaction.csv")
train_identity_path = osp.join(raw_dataset_path, "train_identity.csv")
train_transaction_df = pd.read_csv(train_transaction_path)
train_identity_df = pd.read_csv(train_identity_path)

test_transaction_path = osp.join(raw_dataset_path, "test_transaction.csv")
test_identity_path = osp.join(raw_dataset_path, "test_identity.csv")
test_transaction_df = pd.read_csv(test_transaction_path)
test_identity_df = pd.read_csv(test_identity_path)

In [203]:
transaction_df = pd.concat([train_transaction_df, test_transaction_df], axis=0)
identity_df = pd.concat([train_identity_df, test_identity_df], axis=0)

In [204]:
transaction_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0.0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0.0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0.0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0.0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0.0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [205]:
identity_df.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
# Sort by datetime, later we will use this to make a training and validation split
transaction_df.sort_values("TransactionDT")

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0.0,86400,68.500,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0.0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0.0,86469,59.000,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0.0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0.0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,4170235,NaN,34214279,94.679,C,13832,375.0,185.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506687,4170236,NaN,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506688,4170237,NaN,34214326,49.000,W,16661,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506689,4170238,NaN,34214337,202.000,W,16621,516.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
transaction_df = pd.merge(transaction_df, identity_df, on="TransactionID")
transaction_df

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38
0,2987004,0.0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987008,0.0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987010,0.0,86549,75.887,C,16496,352.0,117.0,mastercard,134.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987011,0.0,86555,16.495,C,4461,375.0,185.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987016,0.0,86620,30.000,H,1790,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286135,4170230,NaN,34214253,10.452,C,5812,408.0,185.0,mastercard,224.0,...,NotFound,NaN,chrome 71.0 for android,NaN,NaN,NaN,F,F,T,F
286136,4170233,NaN,34214271,13.403,C,3154,408.0,185.0,mastercard,224.0,...,Found,NaN,chrome 71.0 for android,NaN,NaN,NaN,F,F,T,F
286137,4170234,NaN,34214277,50.000,H,9002,453.0,150.0,visa,226.0,...,NotFound,iOS 10.3.3,mobile safari 10.0,32.0,1334x750,match_status:2,T,F,F,T
286138,4170236,NaN,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,...,NotFound,NaN,chrome 43.0 for android,NaN,NaN,NaN,F,F,T,F


In [208]:
# Remove the transactions where isFraud is NaN
transaction_df = transaction_df[transaction_df["isFraud"].notna()]
transaction_df

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38
0,2987004,0.0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987008,0.0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987010,0.0,86549,75.887,C,16496,352.0,117.0,mastercard,134.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987011,0.0,86555,16.495,C,4461,375.0,185.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987016,0.0,86620,30.000,H,1790,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,0.0,15810802,48.877,C,12019,305.0,106.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144229,3577526,1.0,15810876,250.000,R,1214,174.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144230,3577529,0.0,15810912,73.838,C,5096,555.0,185.0,mastercard,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144231,3577531,0.0,15810935,400.000,R,6019,583.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
# TODO: Remove this
transaction_df = transaction_df.head(10000)

In [210]:
non_target_node_types = ["card1", "card2", "card3", "card4", "card5", "card6",
                         "ProductCD", "addr1", "addr2", "P_emaildomain", "R_emaildomain"]
## Add Identity columns to node_types
#non_target_node_types.extend(identity_df.columns[1:])

In [211]:
excl_cols = ["TransactionID", "isFraud", "TransactionDT"]
transaction_cat_features = ["M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9",
                            "DeviceType", "DeviceInfo", "id_12", "id_13", "id_14",
                            "id_15", "id_16", "id_17", "id_18", "id_19", "id_20",
                            "id_21", "id_22", "id_23", "id_24", "id_25", "id_26",
                            "id_27", "id_28", "id_29", "id_30", "id_31", "id_32",
                            "id_33", "id_34", "id_35", "id_36", "id_37", "id_38"]
transaction_numeric_features = [column for column in transaction_df.columns
                                if column not in non_target_node_types + excl_cols + transaction_cat_features]

In [212]:
transaction_feat_df = transaction_df[transaction_numeric_features + transaction_cat_features].copy()

In [213]:
transaction_feat_df = transaction_feat_df.fillna(0)

In [215]:
# Process categorical transaction features

# TODO: From pyg
# TODO: I think this can be values instead of 0 and 1

class CategoricalEncoder:

    def __init__(self, key):
        self.key = key

    def __call__(self, df):
        categories = set(
            row[self.key] for _, row in df.iterrows())
        mapping = {cat: i for i, cat in enumerate(categories)}

        x = torch.zeros(len(df), 1)
        for i, row in df.iterrows():
            x[i, 0] = mapping[row[self.key]]
        return x


cat_encoders = [CategoricalEncoder(key) for key in transaction_cat_features]
cat_features = [cat_enc(transaction_feat_df) for cat_enc in cat_encoders]
node_feats = torch.cat(cat_features, dim=-1)
node_feats[0]

tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 303.,   0.,
          0.,   5.,   1.,   0.,  14.,   0.,  22.,  45.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   0.,   9.,  67.,   0.,  13.,   0.,   2.,   1.,
          2.,   2.])

In [216]:
# Process non-categorical transaction features

# TODO: Do something with transactions amounts np.log

def process_val(val):
    if pd.isna(val):
        return 0.0
    return val

other_feats = [
    list(map(process_val, [row[feat] for feat in transaction_numeric_features]))
    for _, row in transaction_feat_df.iterrows()
]
other_feats = torch.tensor(other_feats)
other_feats.shape

torch.Size([10000, 420])

In [217]:
import torch.nn.functional as F

# TODO: node feats is now too big
node_feats = torch.cat((node_feats, other_feats), -1)
node_feats.shape

torch.Size([10000, 458])

### Inspecting the dataset

TODO: Look at a few of the columns - call out important ones we will use for features and labels

TODO: Create validation / test split

NOTES: TODO: Remove

- Nodes are transaction IDs
-

### Creating a PyTorch Geometric dataset

TODO: Create PyTorch Geometric dataset from above - maybe move to a separate script

In [218]:
# TODO: This takes ages
# TODO: Doesn't work if all data is used
# TODO: Tidy this -> make encoders

get_cat_map = lambda vals: {val: idx for idx, val in enumerate(vals)}

def get_edge_list(df, identifier):
    # Find number of unique categories for this node type
    unique_entries = df[identifier].drop_duplicates().dropna()
    # Create a map of category to value
    entry_map = get_cat_map(unique_entries)
    # Create edge list mapping transaction to node type
    edge_list = [[], []]

    for idx, transaction in transaction_df.iterrows():
        node_type_val = transaction[identifier]
        # Don't create nodes for NaN values
        if pd.isna(node_type_val):
            continue
        edge_list[0].append(idx)
        edge_list[1].append(entry_map[node_type_val])
    return torch.tensor(edge_list, dtype=torch.long)


In [219]:
edge_dict = {node_type: get_edge_list(transaction_df, node_type) for node_type in non_target_node_types}

In [220]:
import torch

from torch_geometric.data import HeteroData

data = HeteroData()

In [221]:
data["transaction"].num_nodes = len(transaction_df)
data["transaction"].x = node_feats
data["transaction"].y = torch.tensor(transaction_df['isFraud'], dtype=int)

/tmp/ipykernel_61593/3597386821.py:3: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["transaction"].y = torch.tensor(transaction_df['isFraud'], dtype=int)


In [222]:
for node_type in non_target_node_types:
    data["transaction", "to", node_type].edge_index = edge_dict[node_type]
    data[node_type].num_nodes = edge_dict[node_type][1].max() + 1

In [223]:
data.validate()

True

In [224]:
data

HeteroData(
  transaction={
    num_nodes=10000,
    x=[10000, 458],
    y=[10000]
  },
  card1={ num_nodes=2088 },
  card2={ num_nodes=333 },
  card3={ num_nodes=40 },
  card4={ num_nodes=4 },
  card5={ num_nodes=56 },
  card6={ num_nodes=3 },
  ProductCD={ num_nodes=4 },
  addr1={ num_nodes=177 },
  addr2={ num_nodes=27 },
  P_emaildomain={ num_nodes=57 },
  R_emaildomain={ num_nodes=54 },
  (transaction, to, card1)={ edge_index=[2, 10000] },
  (transaction, to, card2)={ edge_index=[2, 9958] },
  (transaction, to, card3)={ edge_index=[2, 9999] },
  (transaction, to, card4)={ edge_index=[2, 9997] },
  (transaction, to, card5)={ edge_index=[2, 9951] },
  (transaction, to, card6)={ edge_index=[2, 9999] },
  (transaction, to, ProductCD)={ edge_index=[2, 10000] },
  (transaction, to, addr1)={ edge_index=[2, 8399] },
  (transaction, to, addr2)={ edge_index=[2, 8399] },
  (transaction, to, P_emaildomain)={ edge_index=[2, 8982] },
  (transaction, to, R_emaildomain)={ edge_index=[2, 6729] }
)

In [225]:
data.num_nodes

tensor(12843)

### Visualize

In [226]:
import networkx as nx
from matplotlib import pyplot as plt
from torch_geometric.utils import to_networkx

# Convert to homogeneous
data_homogeneous = data.to_homogeneous()
g = to_networkx(data_homogeneous)
# Use node types as colour map
colour_map = data_homogeneous.node_type

# TODO: This maybe?
## Get labels
#labels = {str(idx): val for idx, val in enumerate(data_homogeneous.y)}

# Plot the graph
nx.draw(g, node_color=colour_map, with_labels=True)
plt.show()

RuntimeError: repeats can not be negative

## Preprocess

In [244]:
import torch_geometric.transforms as T

data = T.ToUndirected()(data)
data = T.AddSelfLoops()(data)
data = T.NormalizeFeatures()(data)

data

HeteroData(
  transaction={
    num_nodes=10000,
    x=[10000, 458],
    y=[10000],
    train_mask=[10000],
    val_mask=[10000]
  },
  card1={ num_nodes=2088 },
  card2={ num_nodes=333 },
  card3={ num_nodes=40 },
  card4={ num_nodes=4 },
  card5={ num_nodes=56 },
  card6={ num_nodes=3 },
  ProductCD={ num_nodes=4 },
  addr1={ num_nodes=177 },
  addr2={ num_nodes=27 },
  P_emaildomain={ num_nodes=57 },
  R_emaildomain={ num_nodes=54 },
  (transaction, to, card1)={ edge_index=[2, 10000] },
  (transaction, to, card2)={ edge_index=[2, 9958] },
  (transaction, to, card3)={ edge_index=[2, 9999] },
  (transaction, to, card4)={ edge_index=[2, 9997] },
  (transaction, to, card5)={ edge_index=[2, 9951] },
  (transaction, to, card6)={ edge_index=[2, 9999] },
  (transaction, to, ProductCD)={ edge_index=[2, 10000] },
  (transaction, to, addr1)={ edge_index=[2, 8399] },
  (transaction, to, addr2)={ edge_index=[2, 8399] },
  (transaction, to, P_emaildomain)={ edge_index=[2, 8982] },
  (transaction,

In [245]:
# TODO: Has self loops worked?

### Create dataset splits

TODO: We sort by time above and then select those from more recent as the validation and test

In [246]:
num_nodes_train = int(0.8 * data["transaction"].num_nodes)
data["transaction"].train_mask = torch.zeros(data["transaction"].num_nodes, dtype=bool)
data["transaction"].train_mask[:num_nodes_train] = True
data["transaction"].val_mask = torch.zeros(data["transaction"].num_nodes, dtype=bool)
data["transaction"].val_mask[num_nodes_train:] = True

print(f"Number of training nodes: {data['transaction'].train_mask.sum()}")
print(f"Number of validation nodes: {data['transaction'].val_mask.sum()}")

Number of training nodes: 8000
Number of validation nodes: 2000


In [247]:
data

HeteroData(
  transaction={
    num_nodes=10000,
    x=[10000, 458],
    y=[10000],
    train_mask=[10000],
    val_mask=[10000]
  },
  card1={ num_nodes=2088 },
  card2={ num_nodes=333 },
  card3={ num_nodes=40 },
  card4={ num_nodes=4 },
  card5={ num_nodes=56 },
  card6={ num_nodes=3 },
  ProductCD={ num_nodes=4 },
  addr1={ num_nodes=177 },
  addr2={ num_nodes=27 },
  P_emaildomain={ num_nodes=57 },
  R_emaildomain={ num_nodes=54 },
  (transaction, to, card1)={ edge_index=[2, 10000] },
  (transaction, to, card2)={ edge_index=[2, 9958] },
  (transaction, to, card3)={ edge_index=[2, 9999] },
  (transaction, to, card4)={ edge_index=[2, 9997] },
  (transaction, to, card5)={ edge_index=[2, 9951] },
  (transaction, to, card6)={ edge_index=[2, 9999] },
  (transaction, to, ProductCD)={ edge_index=[2, 10000] },
  (transaction, to, addr1)={ edge_index=[2, 8399] },
  (transaction, to, addr2)={ edge_index=[2, 8399] },
  (transaction, to, P_emaildomain)={ edge_index=[2, 8982] },
  (transaction,

In [248]:
# Number of fraud transactions

percentage_fraud_train = data["transaction"].y[data["transaction"].train_mask].sum() / len(data["transaction"].train_mask)
percentage_fraud_val = data["transaction"].y[data["transaction"].val_mask].sum() / len(data["transaction"].val_mask)
print(f"{percentage_fraud_train = :%}")
print(f"{percentage_fraud_val = :%}")

percentage_fraud_train = 4.020000%
percentage_fraud_val = 0.770000%


## Dataloading

TODO: Is graph too large that need to do some sampling - neighbour sampling

In [249]:
from torch_geometric.data.storage


from draft_fixed_size_neighbour_loader import FixedSizeNeighborLoader

batch_size = 2
num_layers = 2

train_loader_ipu = FixedSizeNeighborLoader(
    data,
    num_neighbors=[15] * num_layers,
    batch_size=batch_size,
    input_nodes=('transaction', data['transaction'].train_mask),
    collater_args={"num_nodes": 10,
                   "num_edges": 50,
                   "exclude_keys": ("batch_size",),
                   "add_masks_to_batch": True},
)

SyntaxError: invalid syntax (2211559872.py, line 1)

In [85]:
sample = next(iter(train_loader_ipu))

In [86]:
from torch_geometric.loader import NeighborLoader

train_loader_cpu = NeighborLoader(
    data,
    num_neighbors=[15] * num_layers,
    batch_size=batch_size,
    input_nodes=('transaction', data['transaction'].train_mask),
)
sample_cpu = next(iter(train_loader_cpu))

In [87]:
sample_cpu["transaction", "to", "product_code"]

{'edge_index': tensor([[0, 1, 2, 3],
        [0, 0, 0, 0]]), 'e_id': tensor([0, 1, 2, 3])}

## Picking the right model

TODO: Describe the task

TODO: Describe different relations type - want something that can do different relations (heterogeneous graph) - RGCN could be a sensible choice
    - but requires weights for every relation type - we have 11 relation types so might be ok

TODO: Try CompGCN
    - Only requires 3 weights - in, out, self loops

In [263]:
import torch.nn as nn
from torch_geometric.nn import SAGEConv, to_hetero

# TODO: Make this work -> Target is a dict


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class Wrapper(torch.nn.Module):

    def __init__(self, hetero_gnn, embedding_size):
        super().__init__()
        self.hetero_gnn = hetero_gnn
        #self.batch_size = batch_size
        self.embedding = nn.ModuleDict({
            node_type: nn.Embedding(data[node_type].num_nodes, embedding_size)
            for node_type in data.node_types
            if node_type != "transaction"
        })

    def forward(self, x_dict, edge_index_dict, target=None):
        for _, _, node_type in edge_index_dict.keys():
            if node_type != "transaction":
                x_dict[node_type] = self.embedding[node_type].weight

        x_dict = self.hetero_gnn(x_dict, edge_index_dict)
        out = x_dict['transaction']
        if self.training:
            #loss = nn.functional.cross_entropy(out[:self.batch_size],
            #                                   target[:self.batch_size])
            loss = nn.functional.cross_entropy(out, target)
            return out, loss
        return out


model = GNN(hidden_channels=64, out_channels=2)
model = to_hetero(model, data.metadata(), aggr="sum")
model = Wrapper(model, embedding_size=128)

with torch.no_grad():  # Initialize lazy modules.
    out_cpu, loss = model(data.x_dict,
                          data.edge_index_dict,
                          target=data["transaction"].y)

out_cpu

transaction torch.Size([10000, 458])
card1 torch.Size([2088, 128])
card2 torch.Size([333, 128])
card3 torch.Size([40, 128])
card4 torch.Size([4, 128])
card5 torch.Size([56, 128])
card6 torch.Size([3, 128])
ProductCD torch.Size([4, 128])
addr1 torch.Size([177, 128])
addr2 torch.Size([27, 128])
P_emaildomain torch.Size([57, 128])
R_emaildomain torch.Size([54, 128])


tensor([[ 2.9439, -1.8834],
        [ 3.1871,  1.6866],
        [ 0.8446, -1.6313],
        ...,
        [ 6.0296, -0.0450],
        [ 3.3921, -0.6793],
        [ 5.6923, -3.1565]])

In [132]:
import poptorch

model.eval()
inf_model = poptorch.inferenceModel(model)
out_ipu = inf_model(data.x_dict,
                    data.edge_index_dict,
                    target=data["transaction"].y)
out_ipu

[13:43:20.635] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[13:43:20.635] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[13:43:20.648] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 3386
[13:43:20.648] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 3387
[13:43:20.648] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 3388
[13:43:20.648] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 3389
[13:43:20.649] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 3390
[13:43:20.649] [poptorch:cpp] [warning] [DISPATCHER] Type coer

tensor([[-0.0126,  0.9219],
        [-0.0322,  0.9219],
        [-0.0322,  0.9219],
        [-0.0322,  0.9219],
        [ 0.0317, -0.0394]])

In [133]:
assert torch.allclose(out_cpu, out_ipu, rtol=1e-05, atol=1e-05)

AssertionError: 

In [104]:
data["transaction"].y

tensor([0, 0, 0, 0, 0])

In [105]:
with torch.no_grad():  # Initialize lazy modules.
    out_cpu, loss = model(data.x_dict,
                          data.edge_index_dict,
                          target=data["transaction"].y)
    
out_cpu

tensor([[-0.2695,  0.2399],
        [-0.2768,  0.2458],
        [-0.2768,  0.2458],
        [-0.2768,  0.2458],
        [-0.2768,  0.2458]])

In [106]:
data

HeteroData(
  transaction={
    num_nodes=5,
    x=[5, 1],
    y=[5],
    train_mask=[5],
    val_mask=[5]
  },
  product_code={ x=[2, 2] },
  purchaser_email_domain={ x=[4, 4] },
  recipient_email_domain={ x=[1, 1] },
  (transaction, to, product_code)={ edge_index=[2, 5] },
  (transaction, to, purchaser_email_domain)={ edge_index=[2, 5] },
  (transaction, to, recipient_email_domain)={ edge_index=[2, 5] },
  (product_code, rev_to, transaction)={ edge_index=[2, 5] },
  (purchaser_email_domain, rev_to, transaction)={ edge_index=[2, 5] },
  (recipient_email_domain, rev_to, transaction)={ edge_index=[2, 5] }
)

In [107]:
import poptorch

model.eval()
inf_model = poptorch.inferenceModel(model)
out_ipu = inf_model(data.x_dict,
                    data.edge_index_dict,
                    target=data["transaction"].y)
out_ipu

[13:11:40.109] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[13:11:40.110] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[13:11:40.125] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1937
[13:11:40.125] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1938
[13:11:40.125] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1939
[13:11:40.126] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1940
[13:11:40.126] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1941
[13:11:40.126] [poptorch:cpp] [warning] [DISPATCHER] Type coer

tensor([[-0.2695,  0.2399],
        [-0.2768,  0.2458],
        [-0.2768,  0.2458],
        [-0.2768,  0.2458],
        [-0.2768,  0.2458]])

In [108]:
assert torch.allclose(out_cpu, out_ipu, rtol=1e-05, atol=1e-05)

In [95]:
sample_cpu["transaction", "to", "product_code"]

{'edge_index': tensor([[0, 1, 2, 3],
        [0, 0, 0, 0]]), 'e_id': tensor([0, 1, 2, 3])}

In [97]:
# TODO: Verify model with padded and unpadded

model = HeteroGNN(
    hidden_channels=64,
    out_channels=2,
    num_layers=num_layers,
    batch_size=batch_size)

with torch.no_grad():  # Initialize lazy modules.
    out_not_padded, loss = model(
        sample_cpu.x_dict,
        sample_cpu.edge_index_dict,
        target=sample_cpu["transaction"].y)

with torch.no_grad():  # Initialize lazy modules.
    out_padded, loss = model(
        sample.x_dict,
        sample.edge_index_dict,
        target=sample["transaction"].y)

assert torch.allclose(out_not_padded, out_padded[:5])

In [ ]:
# TODO: CompGCN?

## Training the model

TODO: Train the model in the normal way

In [98]:
learning_rate = 0.001
num_epochs = 1

In [102]:
# TODO: With sampling

model = HeteroGNN(hidden_channels=64,
                  out_channels=2,
                  num_layers=num_layers,
                  batch_size=batch_size)

sample = next(iter(train_loader_ipu))
with torch.no_grad():  # Initialize lazy modules.
    out_cpu, loss = model(sample.x_dict,
                          sample.edge_index_dict,
                          target=sample["transaction"].y)

model.train()
optimizer = poptorch.optim.Adam(model.parameters(), lr=learning_rate)
training_model = poptorch.trainingModel(model, optimizer=optimizer)

for epoch in range(10):
    total_examples = total_loss = 0
    for batch in train_loader_ipu:
        out, loss = training_model(batch.x_dict,
                                   batch.edge_index_dict,
                                   target=batch['transaction'].y)
        total_examples += batch_size
        total_loss += float(loss) * batch_size
    print(f"Epoch {epoch}, Loss: {total_loss / total_examples}")


[13:10:30.404] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[13:10:30.405] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[13:10:30.418] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1703
[13:10:30.419] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1704
[13:10:30.419] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1705
[13:10:30.419] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1706
[13:10:30.419] [poptorch:cpp] [warning] [DISPATCHER] Type coerced from Long to Int for tensor id 1707
[13:10:30.419] [poptorch:cpp] [warning] [DISPATCHER] Type coer

Epoch 0, Loss: 0.6300453543663025
Epoch 1, Loss: 0.39556291699409485
Epoch 2, Loss: 0.2484317496418953
Epoch 3, Loss: 0.1553347036242485
Epoch 4, Loss: 0.0970383808016777
Epoch 5, Loss: 0.0621356051415205
Epoch 6, Loss: 0.040943074971437454
Epoch 7, Loss: 0.027705077081918716
Epoch 8, Loss: 0.019271982833743095
Epoch 9, Loss: 0.01381582859903574


## Validating our trained model

TODO: Validate the model in the normal way

## Conclusion

> The conclusion to your demo should:
>
> - summarise the main steps that were taken in the demo making clear what
>  your user got to do (similar to steps at the start but more
>  specific, you can link a specific feature/method/class to achieving a specific
>  outcome). (short paragraph: 3-6 sentences)
> - provide resources to go further: these can be links to other tutorials, to
>  documentation, to code examples in the public_examples repo, tech notes, deployments,
>  etc... (2-4 suggestions)
>
> For pointing users to notebooks in the same runtime, point the user to where the file is rather than a link. For example: please see our tutorial, `<folder_name>/<notebook_name>.ipynb`. For relative links the paperspace platform will download the file locally if the machine is running and if the machine is not running will 404. For full path links a new window is opened.